<a href="https://colab.research.google.com/github/AKhilRaghav0/ColabCraft/blob/main/ColabCraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Run a Minecraft Server on Google Colab!**

---

The Colab file provided is designed to facilitate the hosting of a Minecraft server through the Google Compute Engine. Utilizing the storage capabilities of Google Drive, the world file is stored to enable users to revert to previous states of their game. This setup provides an accessible means of hosting a Minecraft server with a convenient and reliable system for managing game data.


In [10]:
import json
import os


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Change directory to the Minecraft server folder on Google Drive
%cd "/content/drive/My Drive/Minecraft-server"
!ls #list the directory contents (to verify that working directory was changed)

# Import config file.
if os.path.isfile("colabconfig.json"):
  colabconfig = json.load(open("colabconfig.json"))
else:
  colabconfig = {"server_type": "generic"} # using default, if config doesn't exists.
  json.dump(colabconfig, open("colabconfig.json",'w'))

# Server jar names.
jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar'}
jar_name = jar_list[colabconfig["server_type"]]

# Java arguments.
if colabconfig["server_type"] == "paper":
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
  server_flags = "" # aiker's flags might negatively impact performance on non-paper servers.
memory_allocation = "-Xmx6144M -Xms6144M"


tunnel_service = "ngrok"
print("Proceeding to use", tunnel_service)       

if tunnel_service == "ngrok":
  !pip -q install pyngrok
  from pyngrok import conf, ngrok
  
  # Ask for the ngrok authtoken
  print("Get your authtoken from https://dashboard.ngrok.com/auth")
  import getpass
  authtoken = getpass.getpass() # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
  ! ngrok authtoken $authtoken # login to ngrok

  # Sets default ngrok region
  conf.get_default().region = 'in'  # Change this to whichever region you want

  # Connect to ngrok
  url = ngrok.connect(25565, 'tcp')
  print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

  print('Starting server...')
  !java $memory_allocation $server_flags -jar $jar_name nogui


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server
banned-ips.json      config	permissions.yml    version_history.json
banned-players.json  eula.txt	plugins		   versions
bukkit.yml	     help.yml	server.jar	   whitelist.json
cache		     libraries	server.properties  world
colabconfig.json     logs	spigot.yml	   world_nether
commands.yml	     ops.json	usercache.json	   world_the_end
Proceeding to use ngrok
Get your authtoken from https://dashboard.ngrok.com/auth
··········
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Your server address is 0.tcp.in.ngrok.io:16208
Starting server...
Starting org.bukkit.craftbukkit.Main
*** Warning, you've not updated in a while! ***
*** Please download a new build as per instructions from https://papermc.io/downloads ***
System Info: Java 17 (OpenJDK 64-Bit Server VM 17.0.6+10-Ubuntu-0ubuntu120.04.1) Host: Linux 5.10.147+

In [1]:
!apt-get install -y openjdk-17-jre-headless
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"


Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei | fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-17-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 43.6 MB of archives.
After this operation, 193 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 openjdk-17-jre-headless amd64 17.0.6+10-0ubuntu1~20.04.1 [43.6 MB]
Fetched 43.6 MB in 2s (18.3 MB/s)
Selecting previously unselected package openjdk-17-jre-headless:amd64.
(Reading database ... 122349 files and directories currently installed.)
Preparing to unpack .../openjdk-17-jre-headless_17.0.6+10-0ubuntu1~20.04.1_amd64.deb ...
Unpacking openjdk-17-jre-headless:amd64 (17.0.6+10-0ubuntu1~20.04.1) ...
Setting up openjdk-17-jre-headless:amd64 (17.

# Make-a-Server

The code below will download a server for you and accept the EULA. After running these scripts, your server will be ready to run.

**Download the Minecraft server**

The code below will download Paper, a high-performance fork of the Vanilla server.
Other server platforms can be used by placing the server.jar in the Drive folder manually.

In [ ]:
# Replace "1.19.2" with your desired server version.
version = '1.19.2'

#Chose server type. Currently available versions: fabric, paper
server_type = 'paper'


from google.colab import drive
import requests
import json


drive.mount('/content/drive')

# Create the directory which will be used for the server
! mkdir "/content/drive/My Drive/Minecraft-server"
%cd "/content/drive/My Drive/Minecraft-server"


a = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version)
#print(a.json()["builds"][-1])
b = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))
#print(b.json()["downloads"]["application"]["name"])
print("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"])

paperURL = "https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]

jar_name = {'paper': 'server.jar', 'fabric': 'fabric-installer.jar'}
url = {
    'paper': (paperURL),
    'fabric': 'https://maven.fabricmc.net/net/fabricmc/fabric-installer/0.7.4/fabric-installer-0.7.4.jar'
    }

print('Downloading to Google Drive...')

r = requests.get(url[server_type])

if r.status_code is 200:
  with open('/content/drive/My Drive/Minecraft-server/' + jar_name[server_type], 'wb') as f:
    f.write(r.content)
else:
  print('Error '+ str(r.status_code) + '! Most likely you entered a unsupported version. Try running the code again if you think that shouldn\'t have happened.')

# Running specific install path.
if server_type == 'fabric':
  !java -jar fabric-installer.jar server -mcversion $version -downloadMinecraft

# Saving config
colabconfig = {"server_type": server_type}
json.dump(colabconfig, open("colabconfig.json",'w'))

print('Done!') # todo: Would show even after erroring.


<>:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:63: SyntaxWarning: "is" with a literal. Did you mean "=="?


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server
https://papermc.io/api/v2/projects/paper/versions/1.19.2/builds/307/downloads/paper-1.19.2-307.jar
Done!


<ipython-input-1-f208500e13ef>:63: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if r.status_code is 200:


**Automatically accept the EULA**

In [5]:
# Please read the file stored in your server folder before running this command. 
# Also, go to https://www.minecraft.net/en-us/eula to read Minecraft's EULA.

# Make sure Drive is mounted
from google.colab import drive
drive.mount('/content/drive')
!ls

%cd "/content/drive/My Drive/Minecraft-server"
!echo "eula=true" >> eula.txt
!echo "online-mode=false" >> server.properties
!cat server.properties

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
banned-ips.json      config	permissions.yml    version_history.json
banned-players.json  eula.txt	plugins		   versions
bukkit.yml	     help.yml	server.jar	   whitelist.json
cache		     libraries	server.properties  world
colabconfig.json     logs	spigot.yml	   world_nether
commands.yml	     ops.json	usercache.json	   world_the_end
/content/drive/My Drive/Minecraft-server
#Minecraft server properties
#Mon Apr 10 04:15:43 UTC 2023
enable-jmx-monitoring=false
rcon.port=25575
level-seed=
gamemode=survival
enable-command-block=false
enable-query=false
generator-settings={}
enforce-secure-profile=true
level-name=world
motd=A Minecraft Server
query.port=25565
pvp=true
generate-structures=true
max-chained-neighbor-updates=1000000
difficulty=easy
network-compression-threshold=256
max-tick-time=60000
require-resource-pack=false
use-native-transport=true
max-players=20
o

In [ ]:
#Get public address (ngrok)
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
## For inspecting the minecraft server directory ##
%cd "/content/drive/My Drive/Minecraft-server"
!ls
